In [1]:
import os
import pandas as pd

# Ruta de la carpeta que contiene los CSVs
ruta_carpeta = './'

# Cargar todos los archivos CSV
dfs = []
for archivo in os.listdir(ruta_carpeta):
    if archivo.endswith('.csv'):
        df = pd.read_csv(os.path.join(ruta_carpeta, archivo))
        df['Fuente'] = archivo  # Agregamos una columna para identificar el origen
        dfs.append(df)

# Combinar todos los DataFrames en uno solo
df_todos = pd.concat(dfs, ignore_index=True)


In [2]:
import pandas as pd
import os
import numpy as np

# Definir las métricas deseadas\ nMETRICAS = ['SMAPE', 'MAE', 'MSE']
METRICAS = ['SMAPE', 'MAE', 'MSE']

def load_results(filename):
    """
    Carga el CSV y extrae el número de intervalo a partir de la columna 'Intervalos Acumulados'.
    Se asume que la cadena tiene el formato 'Intervalos 1 a X'.
    """
    df = pd.read_csv(filename)
    # Extraer el número final del intervalo para usarlo como índice
    df['Intervalo'] = df['Intervalos Acumulados'].str.extract(r'a (\d+)').astype(int)
    df.set_index('Intervalo', inplace=True)
    return df


def extraer_metricas(df, metricas=METRICAS):
    """
    Asegura que el DataFrame tenga las columnas indicadas en 'metricas'.
    Si falta alguna, se agrega con valores NaN.
    Se reordena el DataFrame según el orden especificado en 'metricas'.
    """
    for met in metricas:
        if met not in df.columns:
            df[met] = np.nan
    return df[metricas]


def obtener_tabla(horizonte):
    # Diccionario para almacenar DataFrames de cada modelo
    tablas = {}

    # Archivos para los modelos LSTM
    lstm_files = {
        'LSTM-7': f"lstm_7_{horizonte}.csv",
        'LSTM-14': f"lstm_14_{horizonte}.csv",
        'LSTM-21': f"lstm_21_{horizonte}.csv"
    }
    for key, file in lstm_files.items():
        if os.path.exists(file):
            df = load_results(file)
            tablas[key] = extraer_metricas(df, METRICAS)
        else:
            raise FileNotFoundError(f"Archivo no encontrado: {file}")

    # Archivo para TimesFM
    timesfm_file = f"timesfm_{horizonte}.csv"
    if os.path.exists(timesfm_file):
        df_timesfm = load_results(timesfm_file)
        tablas['TimesFM'] = extraer_metricas(df_timesfm, METRICAS)
    else:
        raise FileNotFoundError(f"Archivo no encontrado: {timesfm_file}")

    # Archivo para Time-MOE
    time_moe_file = f"time-moe_{horizonte}.csv"
    if os.path.exists(time_moe_file):
        df_time_moe = load_results(time_moe_file)
        tablas['Time-MOE'] = extraer_metricas(df_time_moe, METRICAS)
    else:
        raise FileNotFoundError(f"Archivo no encontrado: {time_moe_file}")

    # Archivo para NeuralODE
    neural = f"resultados_neural_ode.csv"
    if os.path.exists(neural):
        df_neural = load_results(neural)
        tablas['Nueral-ODE'] = extraer_metricas(df_neural, METRICAS)
    else:
        raise FileNotFoundError(f"Archivo no encontrado: {neural}")

    # Combinar todos los DataFrames en uno con columnas de MultiIndex
    tabla_resumen = pd.concat(tablas, axis=1)
    tabla_resumen.index.name = 'Intervalo'
    return tabla_resumen


def stack_table(tabla):
    """
    Convierte la tabla de formato ancho (con columnas MultiIndex: Modelo y Métrica)
    a un formato "apilado" donde cada fila representa un intervalo y una métrica,
    y las columnas corresponden a los modelos.
    """
    tabla_apilada = tabla.stack(level=1)
    tabla_apilada.index.names = ['Intervalo', 'Métrica']
    return tabla_apilada


def highlight_min_max(row):
    min_val = row.min()
    max_val = row.max()
    estilos = []
    for v in row:
        if v == min_val and v == max_val:
            estilos.append("")
        elif v == min_val:
            estilos.append("background-color: blue; color: white")
        elif v == max_val:
            estilos.append("background-color: red; color: white")
        else:
            estilos.append("")
    return estilos


def style_stacked(tabla, horizonte):
    return (tabla.style
            .format("{:.2f}")
            .apply(highlight_min_max, axis=1)
            .set_caption(f"Comparación de Modelos por Métrica (por fila: mínimo azul, máximo rojo) - {horizonte} días"))


# Obtener las tablas para los distintos horizontes
tabla_15 = obtener_tabla(15)
tabla_30 = obtener_tabla(30)

# Reorganizar la tabla (apilando las métricas en el índice)
tabla_15_apilada = stack_table(tabla_15)
tabla_30_apilada = stack_table(tabla_30)

# Mostrar las tablas con estilo en Jupyter Notebook
display(style_stacked(tabla_15_apilada, '15'))
display(style_stacked(tabla_30_apilada, '30'))

# Opcional: guardar las tablas a CSV (sin aplicar el estilo)
tabla_15_apilada.to_csv('tabla_apilada_horizonte_15_1.csv')
tabla_30_apilada.to_csv('tabla_apilada_horizonte_30_2.csv')


# USANDO ROBUSET SCALER

C:\Users\criju\AppData\Local\Temp\ipykernel_20632\2305524380.py:85: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  tabla_apilada = tabla.stack(level=1)
C:\Users\criju\AppData\Local\Temp\ipykernel_20632\2305524380.py:85: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  tabla_apilada = tabla.stack(level=1)
